# Crazy Frog modelling 

In [ ]:
# relevant downloads
!pip install boto3
!pip install urllib3==1.25.4
!pip install python-dotenv

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
^C
Traceback (most recent call last):
  File "/root/venv/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionParser
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/parser.py", line 12, in <module>
    from pip._internal.configurati

In [ ]:
# relevant imports
import boto3
import logging
import traceback
from multiprocessing.dummy import Pool  # use threads
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
 

In [ ]:
# DynamoDB connection
load_dotenv()

logging.basicConfig(level=logging.DEBUG, filename='/tmp/crazy.log', filemode="w+",
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

client = boto3.client(
    'dynamodb',
    aws_access_key_id= os.environ["ACCESS_KEY"],
    aws_secret_access_key= os.environ["SECRET_KEY"],
    region_name='eu-central-1'
)
dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=os.environ["ACCESS_KEY"],
    aws_secret_access_key= os.environ["SECRET_KEY"],
    region_name='eu-central-1'
)
ddb_exceptions = client.exceptions

In [ ]:
# function getting all elements from Dynamo table
def get_all():
    items = []
    res = dynamodb.Table('crazy').scan()
    items.extend(res["Items"])
    while 'LastEvaluatedKey' in res:
        res = dynamodb.Table('crazy').scan(ExclusiveStartKey=res['LastEvaluatedKey'])
        items.extend(res["Items"])
    return items

In [1]:
# getting all elements from Crazy DB
data=get_all()
dataF=pd.DataFrame(data)

In [ ]:
# in case you want to save the current database
#dataF.to_csv('database_7_3_23.csv')

# read the csv
#dataF=pd.read_csv('database_7_3_23.csv')

In [ ]:
# backup database
df=dataF

In [ ]:
# keywords
feature_mapping = {"telematics": "telematic", "real time": "real-time", "nb": "nb-iot",
                      "cat m1": "cat-m1", "cat 1": "cat-m1", "ble": "bluetooth", "lte-m": "cat-m1"}


feature_list = ['narrowband', 'sigfox', 'm2m', 'antenna',
        'temperature', 'embedded', 'rfid', 'data', 'security', 'module', 'sensor', 'cat-m1', 'connectivity', 'nb-iot',
        'lpwan', 'track', 'zigbee', 'monitor', 'global', 'logger',
        'mesh', 'updated_at', 'lora', 'sim', '2g', 'remote',
        'gpsr', 'wireless', 'lte', 'alert', 'device', 'cellular',
        'meter', 'plug', 'bandwidth', 'battery', 'network', 'wi-fi',
        'mobile', 'wifi', 'smart', '3g', 'gprs', 'tracker',
        'wirepas', 'unlicensed', 'gsm',
        'telematic', 'deploy', 'bluetooth', 'fleet', 'real-time', 'z-wave',
        'esim', 'iot', 'lorawan', 'ethernet', 'waterproof', '4g', 'm-bus',
        'play', 'grid']

In [ ]:
# based on the feature mapping dictionary similar words are mapped together
def combine_similar(df):

  df_f = df.copy()

  for col in df_f.columns:
    if col in feature_mapping:
      df_f[feature_mapping[col]] += df_f[col]
      df_f.drop([col], axis=1, inplace=True)

  return df_f 

In [ ]:
# print(df['class'].unique())
df = combine_similar(df)

In [ ]:
df.shape

In [ ]:
# create a copy of the dataframe to work on
df_features = df.copy()
df_features = df_features[df_features['exported'] == False]


In [ ]:
# get all lists present in the database
unique_list_id_arr = df_features['list_id'].unique()
print(unique_list_id_arr)
    #map from 
unique_list_id = dict()
idx = 0
for id in unique_list_id_arr:
  if(id != 'noID'):
    unique_list_id[id] = idx
    idx +=1 

## Defining training & test set

Identifying the training dataset as the entries that have been classified and encode the classification values from string to numeric format

In [ ]:
# perform the training only on those companies that have been classified
df_training = df[~df['class'].isnull()].copy()


In [ ]:
# very unbalanced dataset
df_training['class'].value_counts()

In [ ]:
df_training.reset_index(inplace=True)

In [ ]:
# encode classes
for i in range(len(df_training)):
  if df_training['class'][i]=='YES':
    df_training['class'][i]=1
  else:
    df_training['class'][i]=0

In [ ]:
# defining only relevant keywords
relevant = ['narrowband', 'sigfox', 'm2m', 'antenna',
        'temperature', 'embedded', 'rfid', 'data', 'security', 'module', 'sensor', 'cat-m1', 'connectivity', 'nb-iot',
        'lpwan', 'track', 'zigbee', 'monitor', 'global', 'logger',
        'mesh', 'updated_at', 'lora', 'sim', '2g', 'remote',
        'gpsr', 'wireless', 'lte', 'alert', 'device', 'cellular',
        'meter', 'plug', 'bandwidth', 'battery', 'network', 'wi-fi',
        'mobile', 'wifi', 'smart', '3g', 'gprs', 'tracker',
        'wirepas', 'unlicensed', 'gsm',
        'telematic', 'deploy', 'bluetooth', 'fleet', 'real-time', 'z-wave',
        'esim', 'iot', 'lorawan', 'ethernet', 'waterproof', '4g', 'm-bus',
        'play', 'grid','class']

In [ ]:
# keep only the entries where the relevant features are not null
df_training=df_training[relevant].dropna()

In [ ]:
df_training.shape

In [ ]:
# relevant imports for modelling

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
#from skopt import BayesSearchCV
import logging


Define target variable

In [ ]:
#set y
y = df_training['class'].values.astype(float)


In [ ]:
y.shape

Define features for training

In [ ]:
# set features for training
x = df_training[feature_list]

In [ ]:
x.shape

In [ ]:
print("Training classes: ", df_training['class'].unique())

as_dict = df_features.to_dict(orient='records')

Split between train and test set

In [ ]:
# Split the dataset into a train and test set this time using the scaled data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state = 42, stratify=y)

## Undersampling

Given that our dataset is highly unbalanced, undersampling with SMOTE can be used to balance it out 

In [ ]:
!pip install imblearn==0.0

In [ ]:
from imblearn.over_sampling import SMOTE

#perform undersampling
sm = SMOTE(random_state=42)

X_res, y_res = sm.fit_resample(X_train, y_train)

### model tuning with SMOTE
Run this part whenever you want to train again the model and find the new optimal parameters

In [ ]:
#param_grid= {'criterion': ['gini', 'entropy'],'n_estimators':np.arange(1,100, 10), 'min_samples_split': np.arange(2,10, 1) }

In [ ]:
##RF with score >10000, WITH oversampling
#from sklearn.ensemble import RandomForestClassifier

#rdf_c=RandomForestClassifier(criterion='entropy',random_state=0)

#y_res=y_res.values.ravel()
#clf_r = GridSearchCV(rdf_c, param_grid, n_jobs=-1)
#clf_r.fit(X_res, y_res)

#rdf_pred=clf_r.predict(X_test)
#rdf_cm= confusion_matrix(y_test,rdf_pred)
#rdf_ac=accuracy_score(rdf_pred,y_test)
#plt.title("rdf_cm")
#sns.heatmap(rdf_cm,annot=True,fmt="d",cbar=False)
#print('RandomForest_accuracy:',rdf_ac)

In [ ]:
#rdf_cm= confusion_matrix(y_test,rdf_pred)
#rdf_cm=rdf_cm*100/ rdf_cm.astype(np.float).sum(axis=1)
#rdf_ac=accuracy_score(rdf_pred,y_test)
#plt.title("RF with score >10000, WITH oversampling (%)")
#sns.heatmap(rdf_cm,annot=True,cbar=False)
#print('RandomForest_accuracy:',rdf_ac)
##y_axis= true value
##x_axis predicted value

In [ ]:
#print(" Results from Grid Search " )
#print("\n The best estimator across ALL searched params:\n",clf_r.best_estimator_)
#print("\n The best score across ALL searched params:\n",clf_r.best_score_)
#print("\n The best parameters across ALL searched params:\n",clf_r.best_params_)

## model training


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rdf_c=RandomForestClassifier(criterion='gini',random_state=0,min_samples_split= 2, n_estimators= 71)


In [ ]:
rdf_c.fit(X_res, y_res)


In [ ]:
rdf_c

# Lists ready to be scored

In [ ]:
import requests as re
import json
import time
from decimal import Decimal

In [ ]:
query = {
    'token': "token",
    'key': "key"
}

Get current lists in Trello - from "in crazy frog" list

In [ ]:
res = re.get(
   "https://api.trello.com/1/lists/list_id/cards",
   params=query
).json()

Get list name of lists that are "in Crazy Frog" (not classified) from cards

In [ ]:
cards_to_check = []
cards={}
for card in res:
    ress = re.get(
    "https://api.trello.com/1/card/"+card['id'],
    params=query
    ).json()
    name=card['name'] #get name of the cards for the cards_to_check list
    cards_to_check.append(name)
    key= card['name'] # get name of the card for the cards dictionary
    cards[key]=card['id']
print(cards_to_check)
print(cards)
    #cards.append(res)

Find which lists are ready for classification (=fully scraped)

In [ ]:
df_features = dataF.copy()
ready_for_class=[]
still_missing=[]
for list_id in cards_to_check:
    df_features = dataF[dataF['list_id'] == list_id] 
    missing=df_features.narrowband.isna().sum()
    if missing==0:
        ready_for_class.append(list_id) # if no missing values list name is added to the ready_for_class list
    elif missing>0 and missing<4:
        still_missing.append(list_id) # if 1-3 missing values list name is added to the still_missing list 
if len(ready_for_class)>0:
    print('Lists ready for classification:',ready_for_class)
if len(still_missing)>0:
    print('These list still have few nulls, if they are still there next time, check if there are issue in scraping those websites', still_missing)

moving lists to next column

In [ ]:
url = "https://api.trello.com/1/cards"

headers = {
  "Accept": "application/json"
}

Create new cards in "scraped" column

In [ ]:
for i in ready_for_class:
    query = {
        'idList': 'list_id',
        'key': 'key',
        'token': 'token',
        'name': i.replace('.csv',''),

}
    response = re.request(
        "POST",
        url,
        headers=headers,
        params=query)

    print(response.status_code)

Delete previous list

In [ ]:
if response.status_code==200:
    for id_ in ready_for_class:
        delete_id=cards.get(id_)
        url = "https://api.trello.com/1/cards/"+delete_id
        query = {
        'key': 'key',
        'token': 'token'
        }
        del_response = re.request(
        "DELETE",
        url,
        params=query
        )
        if del_response.status_code ==200:
            print('list has been deleted yay')
        else:
            print('list in not in \'In Crazy Frog\'column')

# Set-up processing

From here on we set up all the functions that are going to be applied to our desired list 

## Classify Company

In [ ]:
#company classificaton 
def get_scores(df_features):
    df_features.reset_index(inplace=True)
    for ix in range(df_features.shape[0]):
        try:
            feats=df_features[feature_list].iloc[ix]
            x = np.array(feats).astype(float)
            x=np.round(x,2)
            x = x.reshape(1, -1)
            score=rdf_c.predict_proba(x) #get score
            pred_class=rdf_c.predict(x) #get prediction
            df_features['score'][ix]=score[0][1] #assign probability of class being YES
            df_features['class'][ix]= pred_class.astype(int)
            #print('class:'+ df_features['class'][ix])
            df_features['export_ready'] = True
            df_features['exported'] = True
        except:
            print('Error in some company..')
            try:
                print('continue...')
            except:
                continue
    df_features=df_features.drop('index',axis=1)
    return df_features

## Checking for duplicates
 

In [ ]:
def _domain_https(link):
  domain = 'https://www.'+str(link)
  return domain

In [ ]:
def _domain_http(link):
  domain = 'http://www.'+str(link)
  return domain

In [ ]:
def _domain_www(link):
  domain = 'www.'+str(link)
  #domain = domain.replace('http://','')
  #domain = domain.replace('www.','')
  return domain

In [ ]:
from urllib.request import urlopen
from urllib.error import URLError
from urllib.error import HTTPError
from http import HTTPStatus

Check if domain is already present in HS

In [ ]:
# check if duplicate is present in HS based on propertyName 
def in_HS(prop):
    uri = "https://api.hubapi.com/crm/v3/objects/companies/search"
    headers = {'authorization': 'key','content-type': 'application/json'}
    body = {
        "filterGroups":[
        {
            "filters":[
            {
                "propertyName": 'domain',
                "operator": "EQ",
                "value": prop
            }
            ]
        }
        ]
    }
    res = re.post(uri, headers=headers, data=json.dumps(body)).json()
    time.sleep(0.3)
    try:
        if len(res['results']) > 0:
            print("https://app.hubspot.com/contacts/id/company/"+str(res['results'][0]['id']))
            return True
        else:
    #   print( "Can't match in HubSpot!")
            return False
    except:
        return False

Make domains in their .com version (if not already)

In [ ]:
def internationalise_domain(domain):
    parts = domain.split('.')
    #print(parts[0])
    #print(parts[0], '    ', parts[1])
    try:
        if parts[1] != "com":
            return parts[0]+".com"
    except:
        return domain

check duplicates by name

In [ ]:
def name_in_HS(prop):
    uri = "https://api.hubapi.com/crm/v3/objects/companies/search"
    headers = {'authorization': 'key','content-type': 'application/json'}
    body = {
        "filterGroups":[
        {
            "filters":[
            {
                "propertyName": 'name',
                "operator": "EQ",
                "value": prop
            }
            ]
        }
        ]
    }
    res = re.post(uri, headers=headers, data=json.dumps(body)).json()
    time.sleep(0.3)
    try:
        if len(res['results']) > 0:
            print("https://app.hubspot.com/contacts/id/company/"+str(res['results'][0]['id']))
            return True
        else:
            return False
    except:
        return False

Check if international domain is in HS

In [ ]:
def domain_cleanup(df_features): 

    # check http duplicates
    print('checking for duplicates in the current list...') 
    dom=df_features.domain.iloc[1]
    if 'http' in df_features.domain.iloc[0]:
        df_features['domain'] = df_features.domain.apply(lambda x:x.replace('http://www.',''))
    df_features['InHS'] = df_features.domain.apply(lambda x: in_HS(x))
    
    # check https duplicates
    print('https')
    df_features['domain_https'] = df_features.domain.apply(lambda x: _domain_https(x))
    df_features['InHS_https'] = df_features.domain_https.apply(lambda x: in_HS(x))
    print('http')
    df_features['domain_http'] = df_features.domain.apply(lambda x: _domain_http(x))
    df_features['InHS_http'] = df_features.domain_http.apply(lambda x: in_HS(x))

    # check www duplicates
    print('www')
    df_features['domain_www'] = df_features.domain.apply(lambda x: _domain_www(x))
    df_features['InHS_www'] = df_features.domain_www.apply(lambda x: in_HS(x))
    print('int')
    df_features['int_domain'] = df_features.domain.apply(lambda x: internationalise_domain(x))
    df_features['intInHS'] = df_features.int_domain.apply(lambda x: in_HS(x) if x else None)

    # check name duplicates
    print('name')
    df_features['InHS_name']=df_features.company.apply(lambda x: name_in_HS(x))

    # store duplicates in the same column
    
    i1,i2,i3,i4,i5=-1,-1,-1,-1,-1
    for i in df_features['intInHS']: 
        i1+=1
        if i== True:
            df_features['InHS'][i1]=True 

    for k in df_features['InHS_https']:
        i2+=1
        if k==True:
            df_features['InHS'][i2]=True 

    for j in df_features['InHS_http']:
        i3+=1
        if j==True:
            df_features['InHS'][i3]=True 

    for p in df_features['InHS_www']:
        i4+=1
        if p==True:
            df_features['InHS'][i4]=True 

    for p in df_features['InHS_name']:
        i5+=1
        if p==True:
            df_features['InHS'][i5]=True

Divide between duplicates and new companies 

In [ ]:
def split_df(df_features):
    duplicates = df_features[df_features['InHS']==True] # duplicates dataset
    duplicates.reset_index(inplace=True,drop=True)

    print('There are %d duplicates in the current list' %duplicates.shape[0])
    df_features = df_features[df_features['InHS']==False]
    try:
        df_features.reset_index(inplace=True,drop=True) # non-duplicate dataset
        df_features=df_features.drop('index',axis=1)
    except:
        print('')
        
    return duplicates, df_features

Discard companies if the website is not accessible

In [ ]:
# get the status of a website
def get_website_status(url):
    # handle connection errors
    try:
        # open a connection to the server with a timeout
        with urlopen(url, timeout=3) as connection:
            # get the response code, e.g. 200
            code = connection.getcode()
            c= code
    except HTTPError as e:
        c= e.code
    except URLError as e:
        c= e.reason
    except Exception as e:
        c= e
    print(c)
    if c==200 or c==403:
        return True
    else:
        return False
 
# interpret an HTTP response code into a status
def get_status(code):
    if code == HTTPStatus.OK:
        return 'OK'
    return 'ERROR'
 
# check status of a list of websites
def check_status_urls(urls):
    for url in urls:
        # get the status for the website
        code = get_website_status(url)
        # interpret the status
        status = get_status(code)
        print(status)
        # report status
        #print(f'{url:20s}\t{status:5s}\t{code}')
        if status=='OK':
            return True
        else:
            return False

In [ ]:
def off_domains(df_features):
    print('checking which websites are available...') 
    df_features['formatted_ws'] = df_features.domain.apply(lambda x: 'http://'+str(x)+'/') 
    df_features['ws_on'] = df_features.formatted_ws.apply(lambda x: get_website_status(x)) 
    website_off = df_features[df_features['ws_on']==False] # dataset for non-working websites
    df_features = df_features[df_features['ws_on']==True]  # dataset continuing the processing  
    website_off.reset_index(inplace=True, drop=True)
    try:
        df_features.reset_index(inplace=True, drop=True)
    except:
        print('')
    return df_features, website_off

## update DB with calculated scores

In [ ]:
# update scores in database 

def update_scores(df_features):
    print('Updating scores in DynamoDB...')
    table = dynamodb.Table('crazy')
    for i in range(df_features.shape[0]): 
        comp=df_features['id'][i]
        c=str(df_features['score'][i])
        #df['class'][i]=c
        #print(df['class'][i])
        try:
            response = table.update_item(
                Key= {
                'id': comp
                },
                UpdateExpression= "SET #score= :score",
                ExpressionAttributeNames= {
                    '#score' : 'score'},
                ExpressionAttributeValues={
                    ":score": Decimal(str(df_features['score'][i]))
                }
            )
            #print(response['Attributes'])
        except Exception as e:
            raise Exception(f"Unable to insert filename into dynamodb: {e}") 

## Updates post-scoring

### Preparing csv file

In [ ]:
# creating a csv for the sub lists
def prepare_csv(df_features):
    print('selecting only relevant columns..')
    for_final_csv=['company','class','domain','score','id','list_id']
    df_features=df_features[for_final_csv]
    df_features.insert(loc=2, column='solution', value='') #adding the "solution" column
    #transform classes from arrays to integers
    try:
        df_features['class'] = df_features['class'].apply(lambda x: np.ndarray.item(x))
    except:
        pass
    df_features.reset_index(inplace=True,drop=True)
    print('Exporting a csv file for backup')
    df_features.to_csv(export_specific_id)
    return df_features

### Dividing list in sub-lists

In [ ]:
# find a balanced number of element to have in each list, based on list size
def get_sublist_sizing(df):
    #based on the size of the list decide approach
    print('Getting number of sublists and elements per list')
    list_n=df.shape[0]//150 #max n.of company per list = 150
    if df.shape[0]%150!=0:
        list_n+=1
    items_per_snippet= df.shape[0]//list_n
    return list_n, items_per_snippet

In [ ]:
# divide list in sub-list with balanced number of companies

def create_sublist(df, list_n, items_per_snippet):
    #create dictionary with sub-lists 
    obj = {}
    for i in range(list_n):
        obj['_'+str(i+1)] = df.id[items_per_snippet*i:items_per_snippet*(i+1)]
    #identification for each sub-list
    x=obj.keys()
    update_sublist_aws(x,obj)
    return obj, x

In [ ]:
# update the sub_list field in Dynamo with the sublist n.
def update_sublist_aws(x,obj):
    list_title=export_specific_id.split('.')[0]
    #define sublist for elements in each list
    table = dynamodb.Table('crazy')
    for i in x:
        for comp in obj[i]:
            try:
                response = table.update_item(
                    Key= {
                    'id': comp
                    },
                UpdateExpression= "SET #sub_list_id= :sub_list_id",
                ExpressionAttributeNames= {
                    '#sub_list_id' : 'sub_list_id'},
                ExpressionAttributeValues={
                    ":sub_list_id": list_title+i
                }
                )
            #print(response['Attributes'])
            except Exception as e:
                raise Exception(f"Unable to insert filename into dynamodb: {e}") 

In [ ]:
# update the sub_list field in Dynamo indicating companies in the duplicates dataset are duplicates

def update_duplicates_aws(duplicates):
    table = dynamodb.Table('crazy')
    if duplicates.shape[0]>0:
        duplicates.reset_index(inplace=True)
        for i in range(len(duplicates)): 
            comp=duplicates['id'][i]
            try:
                response = table.update_item(
                    Key= {
                'id': comp
                    },
                    UpdateExpression= "SET #sub_list_id= :sub_list_id",
                    ExpressionAttributeNames= {
                        '#sub_list_id' : 'sub_list_id'},
                    ExpressionAttributeValues={
                        ":sub_list_id": 'duplicate' # specifying it is a duplicate
                    }
                    )
            except Exception as e:
                raise Exception(f"Unable to insert filename into dynamodb: {e}") 

In [ ]:
# update the sub_list field in Dynamo indicating companies in the website_off dataset have websites not working

def update_ws_off_aws(website_off):
    table = dynamodb.Table('crazy')
    if website_off.shape[0]>0:
        website_off.reset_index(inplace=True)
        for i in range(len(website_off)): 
            comp=website_off['id'][i]
            try:
                response = table.update_item(
                    Key= {
                'id': comp
                    },
                    UpdateExpression= "SET #sub_list_id= :sub_list_id",
                    ExpressionAttributeNames= {
                        '#sub_list_id' : 'sub_list_id'},
                    ExpressionAttributeValues={
                        ":sub_list_id": 'offline' # specify the company website does not work
                    }
                    )
            except Exception as e:
                raise Exception(f"Unable to insert filename into dynamodb: {e}") 

check which companies are out of the geographical scope

In [ ]:
ok_countries= ['Albania', 'Algeria', 'Andorra', 'Angola', 'Austria', 'Bahrain', 'Belgium', 'Benin', 'Bosnia and Herzegovina', 'Botswana', 
'Bulgaria', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Comoros', 
'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'DR Congo', 'Egypt', 'Equatorial Guinea', 
'Eritrea', 'Estonia', 'Ethiopia', 'Faroe Islands', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
'Ghana', 'Gibraltar', 'Greece', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Hungary', 'Iceland', 'Iran', 'Iraq', 
'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Ivory Coast', 'Jersey', 'Jordan', 'Kenya', 'Kuwait', 'Latvia',
'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi',
'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mayotte', 'Moldova', 'Monaco', 'Montenegro', 'Morocco', 
'Mozambique', 'Namibia', 'Netherlands', 'Niger', 'Nigeria', 'Norway', 'Oman', 'Palestine', 'Poland', 
'Portugal', 'Qatar', 'Republic of the Congo', 'Reunion', 'Romania', 'Rwanda', 'San Marino', 'Saudi Arabia', 
'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Slovakia', 'Slovenia', 'Somalia', 'South Africa', 'Spain',
'Sudan', 'Sweden', 'Switzerland', 'Syria', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Ukraine', 'United Arab Emirates',
'United Kingdom', 'Vatican City', 'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe', 'Macedonia', 'Kosovo'] 

In [ ]:
# check if the country is in the list of countries within scope
def country_status(country):
    if country  in ok_countries:
        return True
    else:
        return False

In [ ]:
def check_country(df_features):
    df_features['geo_scope_ok'] = df_features.country.apply(lambda x: country_status(x))
    no_scope = df_features[df_features['geo_scope_ok']==False] # dataset of companies out of scope
    df_features = df_features[df_features['geo_scope_ok']==True] # dataset of companies continuing the processing  
    no_scope.reset_index(inplace=True, drop=True)
    df_features.reset_index(inplace=True, drop=True)
    return df_features, no_scope


In [ ]:
# update the sub_list field in Dynamo indicating companies in the no_scope dataset are out of the geographical scope
def update_out_of_scope_aws(no_scope):
    table = dynamodb.Table('crazy')
    if no_scope.shape[0]>0:
        no_scope.reset_index(inplace=True)
        for i in range(len(no_scope)): 
            comp=no_scope['id'][i]
            try:
                response = table.update_item(
                    Key= {
                'id': comp
                    },
                    UpdateExpression= "SET #sub_list_id= :sub_list_id",
                    ExpressionAttributeNames= {
                        '#sub_list_id' : 'sub_list_id'},
                    ExpressionAttributeValues={
                        ":sub_list_id": 'geography' 
                    }
                    )
            except Exception as e:
                raise Exception(f"Unable to insert filename into dynamodb: {e}") 

## Update lists in Trello

In [ ]:
#create new cards
def create_cards(list_title,x,summary):
    url = "https://api.trello.com/1/cards"

    headers = {
    "Accept": "application/json"
    }
    for i in x:
        name=list_title+i
        query = {
            'idList': 'list_id',
            'key': 'key',
            'token': 'token',
            'name': name ,
            'desc': summary, # adding the summary about the number of companies removed and the reason
    }
        response = re.request(
            "POST",
            url,
            headers=headers,
            params=query)

        #print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))
    return response.status_code


In [ ]:
#get current lists in Trello - in #in crazy frog-scraped" list
def get_cards_in_scraped():
    query = {
    'token': "token",
    'key': "key"
    }
    res = re.get(
    "https://api.trello.com/1/lists/list_id/cards",
    params=query
    ).json()
    cards = {}
    for card in res:
        res = re.get(
        "https://api.trello.com/1/card/"+card['id'],
        params=query
        ).json()
        key= res['id']
        cards[key]=res['name']
    print(cards)
    return cards
        #cards.append(res)

In [ ]:
#delete previous list
def delete_card():
    list_title=export_specific_id.split('.')[0]
    cards= get_cards_in_scraped()
    delete_id= list(cards.keys())[list(cards.values()).index(list_title)]
    if list_title in cards.values(): 
        delete_id= list(cards.keys())[list(cards.values()).index(list_title)]
        url = "https://api.trello.com/1/cards/"+delete_id
        query = {
        'key': 'key',
        'token': 'token'
        }
        del_response = re.request(
        "DELETE",
        url,
        params=query
        )

        print(del_response.status_code, 'list has been deleted yay')
    else:
        print('list is not in \'In Crazy Frog - Scraped\'column')
        update_on_slack(list_title, 'is not in \'In Crazy Frog - Scraped\'column, check if list names are correct' )


## Slack notifications

In [ ]:
#sends slack notifications
def update_on_slack(text):
    headers = {
        'Content-type': 'application/json',
    }

    json_data = {
        'text': text,
    }

    response = re.post(
        'https://hooks.slack.com/services/id',
        headers=headers,
        json=json_data,
    )

# List Processing

Input the name of the list you want to make "Ready for analysis"

In [ ]:
#ID_CVR3.csv
#selecting list you want to score

#export_specific_id=input('select list you want to get:')

In [ ]:
#list data-frame
#if export_specific_id != "": 
#  print('specific list...')
#  print(export_specific_id)
#df_features = dataF.copy()

#df_features = dataF[dataF['list_id'] == export_specific_id]
#  default_list_len = 1000

In [ ]:
df_features

,list_id,narrowband,sigfox,m2m,antenna,linkedin,score,country,temperature,embedded,...,attribute,assignee,size,solution,keywords,Processed,sc,NewValue,disq,disqualified_reason
84,ID_HS_OD.csv,0,0,0,0,None,-10000,Spain,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,ID_HS_OD.csv,0,0,0,0,None,-10000,South Korea,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,ID_HS_OD.csv,0,0,0,12,None,-10000,Australia,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
867,ID_HS_OD.csv,0,0,0,0,None,-10000,Sweden,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,ID_HS_OD.csv,0,0,0,0,None,-10000,Denmark,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64762,ID_HS_OD.csv,11,19,1073,0,None,-10000,Netherlands,0,47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64890,ID_HS_OD.csv,0,0,0,0,None,-10000,Sweden,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65567,ID_HS_OD.csv,0,0,5,113,None,-10000,United Arab Emirates,139,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65582,ID_HS_OD.csv,0,1,0,0,None,-10000,Germany,32,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def full_classification(df_features):

    print('shape of the dataset before cleanup is'+ str(df_features.shape))
    initial_shape=str(df_features.shape[0])
    
    #classify each company
    try:
        df_features=get_scores(df_features)
    except:
        update_on_slack('something went wrong in scoring the companies, have a look at the notebook')

    #check wich domain are already present in HubSpot
    try:
        domain_cleanup(df_features)
    except:
        update_on_slack('something went wrong when removing duplicates, have a look at the notebook')

    #separate duplicates from the original dataset 
    duplicates, df_features=split_df(df_features)
    duplicates_shape=str(duplicates.shape[0])

    #remove websites that are not working properly
    try:
        df_features, website_off= off_domains(df_features)
        off_shape=str(website_off.shape[0])
    except:
        update_on_slack('something went wrong in identifying websites not working properly, have a look at the notebook')
    


    # remove companies out of geographical scope
    try:
        df_features,no_scope=check_country(df_features)
        no_scope_shape=str(no_scope.shape[0])
        print('companies out of geographical scope:' + str(no_scope.shape[0]))
    except:
        update_on_slack('something went wrong in identifying companies out of geographical scope, have a look at the notebook')

    #update scores in DynamoDB    
    print('shape of dataset is'+ str(df_features.shape))
    final_shape=str(df_features.shape[0])
    try:
        update_scores(df_features)
    except:
        update_on_slack('something went wrong updating the scores in Dynamo, have a look at the notebook')

    try:
        #remove unnecessary cols and export a csv version of the list
        df_features=prepare_csv(df_features)

        #get number of sublists and dimension of each list 
        list_n, items_per_snippet= get_sublist_sizing(df_features)
    except:
        update_on_slack('something went wrong when splitting the list in sublists, have a look at the notebook')

    #populate sublists and update sublist in DynamoDB
    try:
        obj,x= create_sublist(df_features, list_n, items_per_snippet)
        #update sublist for duplicates in DynamoDB
        update_duplicates_aws(duplicates)
        update_ws_off_aws(website_off)
        #update companies out of geographical scope
        update_out_of_scope_aws(no_scope)        
    except:
        update_on_slack('something went wrong updating the sublists in Dynamo, have a look at the notebook')
        
    
    #create card for each sublist
    list_title=export_specific_id.split('.')[0]
    summary=("Summary for list {0}:\nnumber of elements being processed: {1}\nnumber of duplicates: {2}\n"
    "number of non-accessible websites: {3}\nnumber of companies out of geographical scope: {4}\n"
    "final number of companies: {5}").format(list_title,initial_shape,off_shape,duplicates_shape,no_scope_shape,final_shape)
    response_code= create_cards(list_title,x,summary)
    print(summary)
    #if cards are created, remove card for corresponding list
    if response_code==200:
        delete_card()
        print('List has been processed and now it is ready for analysis')
        msg=list_title+'is ready to be assigned'
        update_on_slack(msg)
    else:
        print('Something went wrong, check again')
        msg=list_title+'something went wrong in creating the cards'
        update_on_slack(msg)

    

In [ ]:
#full_classification(df_features)

In [ ]:
for export_specific_id in ready_for_class:
    df_features = dataF.copy()

    df_features = dataF[dataF['list_id'] == export_specific_id]
    #print(df_features)
    full_classification(df_features)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0e804c14-41ab-4390-bced-8720f7d05820' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>